In [ ]:
%tensorflow_version 1.x


TensorFlow 1.x selected.


In [ ]:
import tensorflow.keras as K
import tensorflow as tf
import numpy as np


In [ ]:
Train, Test = K.datasets.cifar10.load_data()
(x_train, y_train) = Train
(x_test, y_test) = Test

170500096/170498071 [==============================] - 2s 0us/step


In [ ]:
def preprocess_data(X, Y):
    """
    * X is a numpy.ndarray of shape (m, 32, 32, 3) containing
        the CIFAR 10 data, where m is the number of data points
    * Y is a numpy.ndarray of shape (m,) containing the CIFAR 10
        labels for X
        Returns: X_p, Y_p
    * X_p is a numpy.ndarray containing the preprocessed X
    * Y_p is a numpy.ndarray containing the preprocessed Y
    """
    X_p = K.applications.densenet.preprocess_input(X)    
    Y_p = K.utils.to_categorical(y=Y, num_classes=10)
    return (X_p, Y_p)

In [ ]:
xp_train, yp_train = preprocess_data(x_train, y_train)
xp_test, yp_test = preprocess_data(x_test, y_test)

In [ ]:
    lay_init = K.initializers.he_normal()
    # pre_model  
    entrada = K.Input(shape=(32, 32, 3))
    resize = K.layers.Lambda(lambda image: tf.image.resize(image, (155, 155)))(entrada)
    dense169 = K.applications.DenseNet169(include_top=False, weights="imagenet", input_tensor=resize)
    dense169.trainable = False
    out_pre = dense169(resize, training=False)
 
    vector = K.layers.Flatten()(out_pre)
    drop1 = K.layers.Dropout(0.3)(vector)
    norm_lay1 = K.layers.BatchNormalization()(drop1)
    FC1 = K.layers.Dense(units=510, activation='relu', kernel_initializer=lay_init)(norm_lay1)
    norm_lay2 = K.layers.BatchNormalization()(FC1)
    out = K.layers.Dense(units=10, activation='softmax', kernel_initializer=lay_init)(norm_lay2)
 
    model = K.models.Model(inputs=entrada, outputs=out)
    
    learn_dec = K.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
    early = K.callbacks.EarlyStopping(patience=5)
    save = K.callbacks.ModelCheckpoint(filepath='cifar10.h5', save_best_only=True, monitor='val_loss', mode='min')

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x=xp_train, y=yp_train, batch_size=32, epochs=15, verbose=1, validation_data=(xp_test, yp_test), callbacks=[save, early, learn_dec])



Instructions for updating:
If using Keras pass *_constraint arguments to layers.
51879936/51877672 [==============================] - 0s 0us/step
Train on 50000 samples, validate on 10000 samples
Epoch 1/15
50000/50000 [==============================] - 171s 3ms/sample - loss: 0.3975 - acc: 0.8710 - val_loss: 0.2782 - val_acc: 0.9030
Epoch 2/15
50000/50000 [==============================] - 157s 3ms/sample - loss: 0.2193 - acc: 0.9248 - val_loss: 0.2778 - val_acc: 0.9059
Epoch 3/15
50000/50000 [==============================] - 157s 3ms/sample - loss: 0.1536 - acc: 0.9460 - val_loss: 0.2915 - val_acc: 0.9106
Epoch 4/15
50000/50000 [==============================] - 157s 3ms/sample - loss: 0.1129 - acc: 0.9608 - val_loss: 0.2862 - val_acc: 0.9131
Epoch 5/15
50000/50000 [==============================] - 158s 3ms/sample - loss: 0.0566 - acc: 0.9813 - val_loss: 0.2553 - val_acc: 0.9245
Epoch 6/15
50000/50000 [==============================] - 157s 3ms/sample - loss: 0.0387 - acc: 0.9885 -

In [ ]:
K.learning_phase = K.backend.learning_phase
K.is_sparse = K.backend.is_sparse
m = K.models.load_model('cifar10.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
m.evaluate(xp_test, yp_test, verbose=1)

10000/10000 [==============================] - 26s 3ms/sample - loss: 0.2523 - acc: 0.9245


[0.25232971940934656, 0.9245]